In [4]:
from fastai.data.all import *
from fastai.vision.all import *
import pandas as pd
import statistics

In [59]:
df=pd.read_csv('ArtEmisv2.csv')
df

,emotion,art_style,painting
0,fear,Art_Nouveau_Modern,a.y.-jackson_algoma-in-november-1935
1,sadness,Post_Impressionism,a.y.-jackson_barns-1926
2,fear,Post_Impressionism,a.y.-jackson_barns-1926
3,fear,Post_Impressionism,a.y.-jackson_barns-1926
4,contentment,Post_Impressionism,a.y.-jackson_early-spring-quebec-1923
...,...,...,...
237993,anger,Expressionism,zinaida-serebriakova_young-moroccan-1932
237994,sadness,Expressionism,zinaida-serebriakova_young-moroccan-1932
237995,fear,Expressionism,zinaida-serebriakova_young-moroccan-1932
237996,sadness,Expressionism,zinaida-serebriakova_young-moroccan-1932


In [60]:
processed=df.groupby(["art_style","painting"])["emotion"].apply(statistics.mode).reset_index()
processed

,art_style,painting,emotion
0,Abstract_Expressionism,aaron-siskind_chicago-6-1961,fear
1,Abstract_Expressionism,aaron-siskind_feet-102-1957,disgust
2,Abstract_Expressionism,aaron-siskind_kentucky-4-1951,disgust
3,Abstract_Expressionism,aaron-siskind_new-york-24-1988,fear
4,Abstract_Expressionism,aaron-siskind_new-york-city-w-1-1947,fear
...,...,...,...
51514,Ukiyo_e,utagawa-toyokuni_nakamura-utaemon-1,anger
51515,Ukiyo_e,utagawa-toyokuni_segawa-kikunojo-iii-and-bando-mitsugoro-ii-1798,anger
51516,Ukiyo_e,utagawa-toyokuni_the-promenade,fear
51517,Ukiyo_e,utagawa-toyokuni_three-beauties-playing-battledore-and-shuttlecock,sadness


In [61]:
'dataset\\wikiart\\'+str(processed.art_style[2])+'\\'+str(processed.painting[2])+'.jpg'

'dataset\\wikiart\\Abstract_Expressionism\\aaron-siskind_kentucky-4-1951.jpg'

In [63]:
processed['path']= processed.apply(lambda x: 'dataset\\wikiart\\'+x['art_style']+"\\"+ x['painting']+".jpg", axis=1)


In [64]:
processed

,art_style,painting,emotion,path
0,Abstract_Expressionism,aaron-siskind_chicago-6-1961,fear,dataset\wikiart\Abstract_Expressionism\aaron-siskind_chicago-6-1961.jpg
1,Abstract_Expressionism,aaron-siskind_feet-102-1957,disgust,dataset\wikiart\Abstract_Expressionism\aaron-siskind_feet-102-1957.jpg
2,Abstract_Expressionism,aaron-siskind_kentucky-4-1951,disgust,dataset\wikiart\Abstract_Expressionism\aaron-siskind_kentucky-4-1951.jpg
3,Abstract_Expressionism,aaron-siskind_new-york-24-1988,fear,dataset\wikiart\Abstract_Expressionism\aaron-siskind_new-york-24-1988.jpg
4,Abstract_Expressionism,aaron-siskind_new-york-city-w-1-1947,fear,dataset\wikiart\Abstract_Expressionism\aaron-siskind_new-york-city-w-1-1947.jpg
...,...,...,...,...
51514,Ukiyo_e,utagawa-toyokuni_nakamura-utaemon-1,anger,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_nakamura-utaemon-1.jpg
51515,Ukiyo_e,utagawa-toyokuni_segawa-kikunojo-iii-and-bando-mitsugoro-ii-1798,anger,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_segawa-kikunojo-iii-and-bando-mitsugoro-ii-1798.jpg
51516,Ukiyo_e,utagawa-toyokuni_the-promenade,fear,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_the-promenade.jpg
51517,Ukiyo_e,utagawa-toyokuni_three-beauties-playing-battledore-and-shuttlecock,sadness,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_three-beauties-playing-battledore-and-shuttlecock.jpg


In [65]:
imageDB=DataBlock(blocks=(ImageBlock,CategoryBlock),
                  get_x=ColReader('path'),
                  get_y=ColReader('emotion'),
                  item_tfms=Resize(224,ResizeMethod.Pad, PadMode.Zeros))

In [66]:
dls = imageDB.dataloaders(processed,bs=16)
dls.show_batch()

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


FileNotFoundError: [Errno 2] No such file or directory: 'dataset\\wikiart\\Naive_Art_Primitivism\\grã©goire-michonze_field-with-figures.jpg'

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)

In [ ]:
learn.fine_tune(1)

In [67]:
processed[processed['painting']=='grã©goire-michonze_field-with-figures']

,art_style,painting,emotion,path
26794,Naive_Art_Primitivism,grã©goire-michonze_field-with-figures,sadness,dataset\wikiart\Naive_Art_Primitivism\grã©goire-michonze_field-with-figures.jpg


In [52]:
processed=processed.drop([21237])

In [53]:
processed

,art_style,painting,emotion,path
0,Abstract_Expressionism,aaron-siskind_chicago-6-1961,fear,dataset\wikiart\Abstract_Expressionism\aaron-siskind_chicago-6-1961.jpg
1,Abstract_Expressionism,aaron-siskind_feet-102-1957,disgust,dataset\wikiart\Abstract_Expressionism\aaron-siskind_feet-102-1957.jpg
2,Abstract_Expressionism,aaron-siskind_kentucky-4-1951,disgust,dataset\wikiart\Abstract_Expressionism\aaron-siskind_kentucky-4-1951.jpg
3,Abstract_Expressionism,aaron-siskind_new-york-24-1988,fear,dataset\wikiart\Abstract_Expressionism\aaron-siskind_new-york-24-1988.jpg
4,Abstract_Expressionism,aaron-siskind_new-york-city-w-1-1947,fear,dataset\wikiart\Abstract_Expressionism\aaron-siskind_new-york-city-w-1-1947.jpg
...,...,...,...,...
51514,Ukiyo_e,utagawa-toyokuni_nakamura-utaemon-1,anger,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_nakamura-utaemon-1.jpg
51515,Ukiyo_e,utagawa-toyokuni_segawa-kikunojo-iii-and-bando-mitsugoro-ii-1798,anger,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_segawa-kikunojo-iii-and-bando-mitsugoro-ii-1798.jpg
51516,Ukiyo_e,utagawa-toyokuni_the-promenade,fear,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_the-promenade.jpg
51517,Ukiyo_e,utagawa-toyokuni_three-beauties-playing-battledore-and-shuttlecock,sadness,dataset\wikiart\Ukiyo_e\utagawa-toyokuni_three-beauties-playing-battledore-and-shuttlecock.jpg
